In [1]:
!pip install transformers

In [19]:
#load the model
from transformers import AutoModelForCausalLM, AutoTokenizer
import re

# Load the GPT-2 model and tokenizer
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# GPT-2 doesn’t have a pad token by default
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

#check whether it contains python related stuff or not in prompt
def is_python_related(prompt:str)-> bool:
  key=["python", "code", "function", "class", "def", "import", "list", "dict"]
  return any(kw in prompt.lower() for kw in key)


#cleaning the output of LLM model
def clean_output(output:str)-> str:
  clean_lines=[]
  for line in output.split('\n'):
    line=line.rstrip()
    if not line:
      continue
    # Drop instruction-style echoes
    if line.startswith("-") or line.lower().startswith("task:"):
      continue

    # ✅ Fix JS-style imports like "import x from 'x'"
    if re.match(r"^\s*import\s+\w+\s+from\s+['\"]\w+['\"]", line):
        module = re.findall(r"import\s+(\w+)\s+from\s+['\"]\w+['\"]", line)[0]
        clean_lines.append(f"import {module}")
        continue

    # ✅ Keep Python imports (with optional alias, dotted names, and *)
    if re.match(r"^\s*(import\s+[\w\.]+(\s+as\s+\w+)?|from\s+[\w\.]+\s+import\s+[\w\*]+(\s+as\s+\w+)?)", line):
        clean_lines.append(line)
        continue

    # ✅ Assignments
    if re.match(r"^\s*\w+\s*=", line):
        clean_lines.append(line)
        continue

    # ✅ Defs, classes, control flow, return, print
    if re.match(r"^\s*(def |class |print|return|for |if |elif |else:|while )", line):
        clean_lines.append(line)
        continue

    # ✅ Standalone expressions (e.g., arr[::-1], func(x))
    if re.match(r"^\s*[\w\[\]\(\)\:\+\-\*/\.,'\"]+$", line):
        clean_lines.append(line)
        continue

  code = "\n".join(clean_lines).strip()
  # Collapse multiple blank lines
  code = re.sub(r"\n\s*\n+", "\n", code)

  return code if code else output.strip()  # fallback

def generate_response(prompt:str,max_length:int=150)->str:
  if not is_python_related(prompt):
    return "I can only answer python related questions"

  inputs = tokenizer.encode(prompt, return_tensors="pt")

  outputs = model.generate(
        inputs,
        max_length=max_length,
        do_sample=False,        # deterministic
        num_beams=1,            # greedy
        repetition_penalty=1.2, # discourage echo
        pad_token_id=tokenizer.eos_token_id
    )

  response = tokenizer.decode(outputs[0], skip_special_tokens=True)

  #loom for the output and remove the prompt

  if prompt in response:
    response = response.split(prompt, 1)[-1].strip()




  return clean_output(response)



if __name__=="__main__":
  prompt_list=["Write python code that reverses the list","write python code to only generate top 50 prime number", "What is photosynthesis"]


  for prompt in prompt_list:
    print(f"Prompt : {prompt}\n")
    print("Response :",generate_response(prompt))




Prompt : Write python code that reverses the list

Response : of values in a dictionary.
The following example shows how to use this method: >>> from django import BeautifulSoup def get_dict ( self ): return dict . append ( 'name' , name ) # This is what we want for our class, so it's not really needed here ... end
Prompt : write python code to only generate top 50 prime number

Response : values.
Prompt : What is photosynthesis

Response : I can only answer python related questions
